# Atelier 4 : Clustering


L'objectif de cet atelier est de detecter le plagiarisme dans une collection de documents texte en regroupant dans les mêmes clusters les documents texte qui était recopier à partir du même document original.
On va reprendre le même dataset des ateliers 1 et 2.

# 1.	Preparation du DataSet

Écrire le code permettant de récupérer le dataset du plagiarisme traité dans les ateliers précédents et appliquer les prétraitements nécessaires.

In [4]:
import os
import glob
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # Tokenization
    words = word_tokenize(text)

    # Lemmatization and remove stopwords
    processed_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]

    return ' '.join(processed_words)

def get_corpus(folder_path):
    corpus = []

    # Utiliser la librairie glob pour récupérer les documents dans une liste de listes
    for file_path in glob.glob(os.path.join(folder_path, '*.txt')):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            preprocessed_content = preprocess_text(content)
            corpus.append(preprocessed_content)

    return corpus

# Utilisation de la fonction
folder_path = "plagitat_corpus/plagitat_corpus/original"
corpus_original = get_corpus(folder_path)

# Affichage du corpus
print(corpus_original)

['probability theory bayes theorem often called bayes law rev thomas bayes relates conditional marginal probability two random event often used compute posterior probability given observation example patient may observed certain symptom bayes theorem used compute probability proposed diagnosis correct given observation see example 2 formal theorem bayes theorem valid common interpretation probability however play central role debate around foundation statistic frequentist bayesian interpretation disagree way probability assigned application frequentists assign probability random event according frequency occurrence subset population proportion whole bayesians describe probability term belief degree uncertainty article bayesian probability frequentist probability discus debate greater detail bayes theorem relates conditional marginal probability event b b probability p p b p p b term bayes theorem conventional name p prior probability marginal probability prior sense take account inform

In [5]:
folder_path = "plagitat_corpus/plagitat_corpus/PA"
corpus_PA = get_corpus(folder_path)
print(corpus_PA)

folder_path = "plagitat_corpus/plagitat_corpus/PB"
corpus_PB = get_corpus(folder_path)
print(corpus_PB)

folder_path = "plagitat_corpus/plagitat_corpus/PC"
corpus_PC = get_corpus(folder_path)
print(corpus_PC)

folder_path = "plagitat_corpus/plagitat_corpus/PD"
corpus_PD = get_corpus(folder_path)
print(corpus_PD)

folder_path = "plagitat_corpus/plagitat_corpus/PE"
corpus_PE = get_corpus(folder_path)
print(corpus_PE)

['inheritance basic concept programming basic idea create new class add extra detail existing class done allowing new class reuse method variable existing class new method class added specialise new class inheritance model relationship entity object example postgraduate undergraduate kind student kind relationship visualised tree structure student would general root node postgraduate undergraduate would specialised extension student node child node relationship student would known superclass parent class whereas postgraduate would known subclass child class postgraduate class extends student class inheritance occur several layer visualised would display larger tree structure example could extend postgraduate node adding two extra extended class called msc student phd student type student kind postgraduate student would mean msc student phd student class would inherit method variable postgraduate student class', 'dynamic programming algorithm design technique used optimisation problem m

# 2. Vectorisation

## 2.1 TF-IDF

Définir le traitement permettant de récupérer la représentation vectorielle TFIDF des différents documents du corpus.

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorsTFIDF(corpus_lemmatized, max_features=None):
    vectorizer = TfidfVectorizer(max_features=max_features)
    vectors = vectorizer.fit_transform(corpus_lemmatized)
    return vectors

vectors_original = vectorsTFIDF(corpus_original)
print(vectors_original)

  (0, 476)	0.04539921276717383
  (0, 288)	0.04539921276717383
  (0, 293)	0.025577124562713233
  (0, 108)	0.030404381897685743
  (0, 165)	0.030404381897685743
  (0, 218)	0.04539921276717383
  (0, 81)	0.03662779900386729
  (0, 280)	0.04539921276717383
  (0, 11)	0.04539921276717383
  (0, 482)	0.030404381897685743
  (0, 436)	0.04539921276717383
  (0, 477)	0.03662779900386729
  (0, 104)	0.030404381897685743
  (0, 105)	0.030404381897685743
  (0, 21)	0.021632968134379205
  (0, 208)	0.030404381897685743
  (0, 10)	0.04539921276717383
  (0, 453)	0.030404381897685743
  (0, 405)	0.03662779900386729
  (0, 332)	0.1361976383015215
  (0, 273)	0.030404381897685743
  (0, 88)	0.04539921276717383
  (0, 110)	0.03662779900386729
  (0, 180)	0.04539921276717383
  (0, 118)	0.04539921276717383
  :	:
  (4, 478)	0.017086068405083678
  (4, 362)	0.020583384375768794
  (4, 480)	0.017086068405083678
  (4, 19)	0.10291692187884398
  (4, 308)	0.020583384375768794
  (4, 506)	0.034172136810167356
  (4, 383)	0.017086068405

In [34]:
vectors_PA = vectorsTFIDF(corpus_PA)
vectors_PB = vectorsTFIDF(corpus_PB)
vectors_PC = vectorsTFIDF(corpus_PC)
vectors_PD = vectorsTFIDF(corpus_PD)
vectors_PE = vectorsTFIDF(corpus_PE)

## 2.2 SVD

Définir le traitement permettant de récupérer la représentation vectorielle des différents documents du corpus en se basant sur l'approche SVD.

In [8]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

def docVectSVD(corpus_lemmatized, n_components=None):
    # Récupérer le vocabulaire du corpus
    vocabulary = set(' '.join(corpus_lemmatized).split())

    # Créer la matrice des cooccurrences des termes
    term_document_matrix = np.zeros((len(vocabulary), len(corpus_lemmatized)))

    term_to_index = {term: idx for idx, term in enumerate(vocabulary)}
    for doc_idx, doc in enumerate(corpus_lemmatized):
        for term in doc.split():
            term_idx = term_to_index.get(term)
            if term_idx is not None:
                term_document_matrix[term_idx, doc_idx] += 1

    # Appliquer la réduction SVD
    if n_components is None:
        n_components = min(len(corpus_lemmatized), len(vocabulary))

    svd = TruncatedSVD(n_components=n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    reduced_matrix = lsa.fit_transform(term_document_matrix)

    # Générer le dictionnaire terme/vecteur
    term_vectors = {term: reduced_matrix[term_to_index[term]] for term in vocabulary}

    # Définir les traitements pour récupérer la représentation vectorielle des documents
    # en se basant sur l'approche SVD (la moyenne des vecteurs composant chaque document).
    document_vectors = []
    for doc in corpus_lemmatized:
        terms_in_doc = [term for term in doc.split() if term in term_to_index]
        if terms_in_doc:
            doc_vector = np.mean([term_vectors[term] for term in terms_in_doc], axis=0)
        else:
            doc_vector = np.zeros(n_components)  # Si le document est vide, vecteur nul
        document_vectors.append(doc_vector)

    return document_vectors

In [9]:
vectorsSVD_original = docVectSVD(corpus_original)
print(vectorsSVD_original)

vectorsSVD_PA = docVectSVD(corpus_PA)
print(vectorsSVD_PA)

vectorsSVD_PB = docVectSVD(corpus_PB)
print(vectorsSVD_PB)

vectorsSVD_PC = docVectSVD(corpus_PC)
print(vectorsSVD_PC)

vectorsSVD_PD = docVectSVD(corpus_PD)
print(vectorsSVD_PD)

vectorsSVD_PE = docVectSVD(corpus_PE)
print(vectorsSVD_PE)

[array([ 0.16705049,  0.38618473,  0.81017782, -0.03415942,  0.01271672]), array([ 0.11534645,  0.865914  , -0.28553656, -0.08499046, -0.0175911 ]), array([ 0.15765308,  0.27029216,  0.0284052 ,  0.77873608, -0.32099632]), array([ 0.13766803,  0.24255127, -0.00637064,  0.35509793,  0.75942647]), array([ 0.91787784, -0.02475561, -0.02947997,  0.01801906,  0.02007777])]
[array([ 0.92175437, -0.10385839, -0.02532117, -0.12968382,  0.08400138]), array([ 0.12599687,  0.27458787,  0.86778335, -0.15675735, -0.07439142]), array([ 0.18126364,  0.87400857, -0.22747166, -0.15450338, -0.04418134]), array([ 0.32322629,  0.18054404,  0.08066651,  0.68743909, -0.45058114]), array([0.07528628, 0.31505362, 0.14119078, 0.44775424, 0.72704356])]
[array([ 0.48976637,  0.762077  , -0.2442926 , -0.04212694, -0.0467445 ]), array([ 0.22838398,  0.26163163,  0.84386199, -0.12620908, -0.0113245 ]), array([ 0.8375624 , -0.43489187, -0.02185791, -0.01463372, -0.03164986]), array([ 0.15079955,  0.09517595,  0.1597

## 2.3 Word2Vec

Definir le traitement permettant de récupérer la representation vectorielle des differents documents du corpus en se basant sur l'approche word2vec.

Tester avec le pretrained glove-wiki-gigaword-50 et avec un apprentissage à partir du corpus.


In [21]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

def pretrainedDocWV(corpus_lemmatized, pretrained_model_path=None, training=False):
    if pretrained_model_path and not training:
        # Utiliser le modèle pré-entraîné
        wv_model = KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)
    else:
        # Entraîner un nouveau modèle
        sentences = [doc.split() for doc in corpus_lemmatized]
        wv_model = Word2Vec(sentences, vector_size=50, window=5, min_count=1, workers=4)

    # Définir la représentation vectorielle de chaque document
    document_vectors = []
    for doc in corpus_lemmatized:
        terms_in_doc = [term for term in doc.split() if term in wv_model.wv]
        if terms_in_doc:
            doc_vector = np.mean([wv_model.wv[term] for term in terms_in_doc], axis=0)
        else:
            doc_vector = np.zeros(wv_model.vector_size)  # Si le document est vide, vecteur nul
        document_vectors.append(doc_vector)

    return document_vectors, wv_model

# Exemple d'utilisation avec un modèle pré-entraîné
pretrained_model_path = "chemin/vers/glove-wiki-gigaword-50.bin"
vectors_pretrained, _ = pretrainedDocWV(corpus_original)

# Exemple d'utilisation avec un modèle entraîné sur le corpus
vectors_trained_original, wv_model_trained = pretrainedDocWV(corpus_original, training=True)


In [18]:
vectors_trained_PA, wv_model_trained = pretrainedDocWV(corpus_PA, training=True)
vectors_trained_PB, wv_model_trained = pretrainedDocWV(corpus_PB, training=True)
vectors_trained_PC, wv_model_trained = pretrainedDocWV(corpus_PC, training=True)
vectors_trained_PD, wv_model_trained = pretrainedDocWV(corpus_PD, training=True)
vectors_trained_PE, wv_model_trained = pretrainedDocWV(corpus_PE, training=True)

In [22]:
# Afficher les vecteurs pour le corpus original après entraînement
print("Vecteurs pour le corpus original après entraînement:")
print(vectors_trained_original)

# Afficher les vecteurs pour le corpus PA après entraînement
print("Vecteurs pour le corpus PA après entraînement:")
print(vectors_trained_PA)

# Afficher les vecteurs pour le corpus PB après entraînement
print("\nVecteurs pour le corpus PB après entraînement:")
print(vectors_trained_PB)

# Afficher les vecteurs pour le corpus PC après entraînement
print("\nVecteurs pour le corpus PC après entraînement:")
print(vectors_trained_PC)

# Afficher les vecteurs pour le corpus PD après entraînement
print("\nVecteurs pour le corpus PD après entraînement:")
print(vectors_trained_PD)

# Afficher les vecteurs pour le corpus PE après entraînement
print("\nVecteurs pour le corpus PE après entraînement:")
print(vectors_trained_PE)

Vecteurs pour le corpus original après entraînement:
[array([ 7.7796291e-04,  2.1346393e-03, -1.0715458e-04,  6.5533788e-04,
        1.7371950e-03, -3.0014627e-03,  3.2307353e-04,  2.9835461e-03,
       -1.4393147e-03, -2.3954043e-03, -4.4396287e-04, -2.0159786e-03,
        1.9644611e-03,  1.6215449e-03, -2.2760394e-03,  2.3470873e-03,
        3.5624353e-03,  7.8046764e-04, -4.1955840e-03, -1.1044285e-03,
       -9.5017276e-05,  7.2493631e-04,  7.2196155e-04, -2.8332628e-03,
       -4.1343048e-04,  2.7577911e-04, -1.5777489e-03,  5.5959012e-04,
       -1.2889060e-03,  1.3968281e-03,  2.2905481e-03, -2.8022185e-03,
       -1.4305813e-03,  1.6017816e-03,  3.5599111e-05,  1.8707250e-03,
        2.0658958e-03,  1.8220952e-04,  3.1053717e-03,  1.3142313e-03,
       -3.8694434e-05,  5.3424935e-04,  8.7094511e-04, -7.2161652e-05,
        3.0090685e-03,  1.0469957e-03,  2.8832864e-03,  1.0297146e-03,
        3.1811746e-03,  2.8060671e-04], dtype=float32), array([-1.0236360e-03, -5.3664076e-04,

## 2.4 Doc2Vec

Définir le traitement permettant de récupérer la representation vectorielle doc2vec des différents documents du corpus

In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def vectorsDoc2Vec(corpus_lemmatized, vector_size=50, window=5, min_count=1, epochs=10):
    # Préparer les données pour Doc2Vec
    tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()), tags=[str(i)]) for i, doc in enumerate(corpus_lemmatized)]

    # Initialiser le modèle Doc2Vec
    model = Doc2Vec(vector_size=vector_size, window=window, min_count=min_count, workers=4, epochs=epochs)

    # Construire le vocabulaire
    model.build_vocab(tagged_data)

    # Entraîner le modèle
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

    # Obtenir les vecteurs des documents
    document_vectors = [model.dv[str(i)] for i in range(len(corpus_lemmatized))]

    return document_vectors, model


In [11]:
vectors_doc2vec_original, doc2vec_model = vectorsDoc2Vec(corpus_original)
vectors_doc2vec_PA, doc2vec_model = vectorsDoc2Vec(corpus_PA)
vectors_doc2vec_PB, doc2vec_model = vectorsDoc2Vec(corpus_PB)
vectors_doc2vec_PC, doc2vec_model = vectorsDoc2Vec(corpus_PC)
vectors_doc2vec_PD, doc2vec_model = vectorsDoc2Vec(corpus_PD)
vectors_doc2vec_PE, doc2vec_model = vectorsDoc2Vec(corpus_PE)

In [12]:
# Afficher les vecteurs pour le corpus original
print("Vecteurs pour le corpus original:")
print(vectors_doc2vec_original)

# Afficher les vecteurs pour le corpus PA
print("\nVecteurs pour le corpus PA:")
print(vectors_doc2vec_PA)

# Afficher les vecteurs pour le corpus PB
print("\nVecteurs pour le corpus PB:")
print(vectors_doc2vec_PB)

# Afficher les vecteurs pour le corpus PC
print("\nVecteurs pour le corpus PC:")
print(vectors_doc2vec_PC)

# Afficher les vecteurs pour le corpus PD
print("\nVecteurs pour le corpus PD:")
print(vectors_doc2vec_PD)

# Afficher les vecteurs pour le corpus PE
print("\nVecteurs pour le corpus PE:")
print(vectors_doc2vec_PE)

Vecteurs pour le corpus original:
[array([-0.03902468, -0.02202072, -0.02806747, -0.00201029,  0.01784034,
        0.0106988 , -0.00763914,  0.01972198, -0.05933684,  0.02588152,
        0.01368911,  0.02985927, -0.00523314, -0.03616172,  0.00416654,
       -0.03886116,  0.00792362,  0.01958848, -0.06491467, -0.03523985,
       -0.04046679,  0.00497833, -0.02144508,  0.0077758 ,  0.00504583,
       -0.00715837, -0.05006787, -0.0523333 , -0.00968706, -0.02815338,
        0.04362073,  0.03395879, -0.04273582, -0.01772884, -0.01561669,
        0.04126262,  0.0023639 , -0.0401254 ,  0.00861472, -0.01760132,
        0.00549841,  0.0080797 ,  0.01243852, -0.04558023,  0.01067955,
        0.01736089,  0.0178703 , -0.0044646 ,  0.03617565, -0.02809025],
      dtype=float32), array([-0.06311646, -0.02162181, -0.03288555, -0.05671869,  0.01449849,
        0.04247233,  0.02334724,  0.07093251, -0.09256854,  0.06706595,
        0.02343508,  0.0610225 ,  0.02086173, -0.06753781,  0.02590375,
      

# 3. Clustering avec K-means

## 3.1 k-means avec TFIDF

Ci-dessous un exemple d'application du k-means en se basant sur le representation TFIDF.  

In [ ]:
#K-MEANS CLUSTERING

#fit(X[, y, sample_weight])          : Compute k-means clustering.
#predict(X[, sample_weight])         : Predict the closest cluster each sample in X belongs to.
#fit_predict(X[, y, sample_weight])  :Compute cluster centers and predict cluster index for each sample.

#fit(X[, y, sample_weight])          : Compute k-means clustering.
#transform(X)                        :Transform X to a cluster-distance space.
#fit_transform(X[, y, sample_weight]):Compute clustering and transform X to cluster-distance space.

#score(X[, y, sample_weight])        : Opposite of the value of X on the K-means objective.


from sklearn.cluster import KMeans
import numpy as np

#initialisation automatique
#km = KMeans(n_clusters=5, init='k-means++',n_init=50)


#predetermination des centroids
centroids=np.array([[-0.10512687, -0.35341345],[-0.24220548,  0.32397296],[-0.03986034,  0.09208828],[0.72163033, 0.16627089],[0.72163033, 0.16627089]])
km = KMeans(n_clusters=5, init=centroids,n_init=50)
print("Clustering sparse data with %s" % km)

vect=vectorsTFIDF(get_corpus('/Users/rachad/Desktop/corpus/*'))

#reduction
import numpy
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
vect = pca.fit_transform(vect.toarray())

km.fit(vect)
y_means = km.predict(vect)



## 3.2 k-means avec SVD

In [28]:
from sklearn.cluster import KMeans
import numpy as np

# Concaténer les vecteurs SVD des différents corpus
all_vectors_svd = np.vstack([vectorsSVD_original, vectorsSVD_PA, vectorsSVD_PB, vectorsSVD_PC, vectorsSVD_PD, vectorsSVD_PE])

# Définir le nombre de clusters (vous pouvez ajuster ce nombre en fonction de vos besoins)
num_clusters_svd = 6

# Appliquer l'algorithme K-Means sur les vecteurs SVD
kmeans_svd = KMeans(n_clusters=num_clusters_svd, random_state=42)
kmeans_svd.fit(all_vectors_svd)

# Ajouter les étiquettes de cluster aux vecteurs SVD
all_vectors_svd_with_labels = np.hstack([all_vectors_svd, kmeans_svd.labels_.reshape(-1, 1)])

# Extraire les étiquettes de cluster SVD pour chaque corpus
labels_svd_original = kmeans_svd.labels_[:len(vectorsSVD_original)]
labels_svd_PA = kmeans_svd.labels_[len(vectorsSVD_original):len(vectorsSVD_original) + len(vectorsSVD_PA)]
labels_svd_PB = kmeans_svd.labels_[len(vectorsSVD_original) + len(vectorsSVD_PA):len(vectorsSVD_original) + len(vectorsSVD_PA) + len(vectorsSVD_PB)]
labels_svd_PC = kmeans_svd.labels_[len(vectorsSVD_original) + len(vectorsSVD_PA) + len(vectorsSVD_PB):len(vectorsSVD_original) + len(vectorsSVD_PA) + len(vectorsSVD_PB) + len(vectorsSVD_PC)]
labels_svd_PD = kmeans_svd.labels_[len(vectorsSVD_original) + len(vectorsSVD_PA) + len(vectorsSVD_PB) + len(vectorsSVD_PC):len(vectorsSVD_original) + len(vectorsSVD_PA) + len(vectorsSVD_PB) + len(vectorsSVD_PC) + len(vectorsSVD_PD)]
labels_svd_PE = kmeans_svd.labels_[len(vectorsSVD_original) + len(vectorsSVD_PA) + len(vectorsSVD_PB) + len(vectorsSVD_PC) + len(vectorsSVD_PD):]

# Afficher les étiquettes de cluster SVD pour chaque corpus
print("\nCluster Labels pour le corpus original avec SVD:")
print(labels_svd_original)

print("\nCluster Labels pour le corpus PA avec SVD:")
print(labels_svd_PA)

print("\nCluster Labels pour le corpus PB avec SVD:")
print(labels_svd_PB)

print("\nCluster Labels pour le corpus PC avec SVD:")
print(labels_svd_PC)

print("\nCluster Labels pour le corpus PD avec SVD:")
print(labels_svd_PD)

print("\nCluster Labels pour le corpus PE avec SVD:")
print(labels_svd_PE)

/home/olivier/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fa6c99cf6a0>
Traceback (most recent call last):
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
   


Cluster Labels pour le corpus original avec SVD:
[3 5 4 0 1]

Cluster Labels pour le corpus PA avec SVD:
[1 3 5 4 0]

Cluster Labels pour le corpus PB avec SVD:
[2 3 1 4 0]

Cluster Labels pour le corpus PC avec SVD:
[3 1 2 0 4]

Cluster Labels pour le corpus PD avec SVD:
[4 3 0 1 2]

Cluster Labels pour le corpus PE avec SVD:
[0 5 4 3 1]


## 3.3 k-means avec Word2Vec


In [23]:
from sklearn.cluster import KMeans
import numpy as np

# Concaténer les vecteurs pour le corpus original après entraînement
all_vectors_trained = np.vstack((vectors_trained_PA, vectors_trained_PB, vectors_trained_PC, vectors_trained_PD, vectors_trained_PE))

# Spécifier le nombre de clusters
num_clusters = 3  # Vous pouvez ajuster ce nombre en fonction de votre cas d'utilisation

# Appliquer K-Means
kmeans_trained = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_trained.fit(all_vectors_trained)

# Obtenir les étiquettes de cluster pour chaque document
cluster_labels_trained_PA = kmeans_trained.predict(vectors_trained_PA)
cluster_labels_trained_PB = kmeans_trained.predict(vectors_trained_PB)
cluster_labels_trained_PC = kmeans_trained.predict(vectors_trained_PC)
cluster_labels_trained_PD = kmeans_trained.predict(vectors_trained_PD)
cluster_labels_trained_PE = kmeans_trained.predict(vectors_trained_PE)

# Afficher les étiquettes de cluster pour le corpus PA
print("Cluster Labels pour le corpus PA:")
print(cluster_labels_trained_PA)

# Afficher les étiquettes de cluster pour le corpus PB
print("\nCluster Labels pour le corpus PB:")
print(cluster_labels_trained_PB)

# Afficher les étiquettes de cluster pour le corpus PC
print("\nCluster Labels pour le corpus PC:")
print(cluster_labels_trained_PC)

# Afficher les étiquettes de cluster pour le corpus PD
print("\nCluster Labels pour le corpus PD:")
print(cluster_labels_trained_PD)

# Afficher les étiquettes de cluster pour le corpus PE
print("\nCluster Labels pour le corpus PE:")
print(cluster_labels_trained_PE)

/home/olivier/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fa6c90abd80>
Traceback (most recent call last):
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
   

ValueError: Buffer dtype mismatch, expected 'const double' but got 'float'

In [25]:
# Convertir les vecteurs Word2Vec pré-entraînés en float64
vectors_trained_PA = np.array(vectors_trained_PA).astype(np.float64)
vectors_trained_PB = np.array(vectors_trained_PB).astype(np.float64)
vectors_trained_PC = np.array(vectors_trained_PC).astype(np.float64)
vectors_trained_PD = np.array(vectors_trained_PD).astype(np.float64)
vectors_trained_PE = np.array(vectors_trained_PE).astype(np.float64)

# Concaténer les vecteurs pour le corpus original après entraînement
all_vectors_trained = np.vstack((vectors_trained_PA, vectors_trained_PB, vectors_trained_PC, vectors_trained_PD, vectors_trained_PE))

# Appliquer K-Means
kmeans_trained = KMeans(n_clusters=num_clusters, random_state=42)
kmeans_trained.fit(all_vectors_trained)

# Obtenir les étiquettes de cluster pour chaque document
cluster_labels_trained_PA = kmeans_trained.predict(vectors_trained_PA)
cluster_labels_trained_PB = kmeans_trained.predict(vectors_trained_PB)
cluster_labels_trained_PC = kmeans_trained.predict(vectors_trained_PC)
cluster_labels_trained_PD = kmeans_trained.predict(vectors_trained_PD)
cluster_labels_trained_PE = kmeans_trained.predict(vectors_trained_PE)

# Afficher les étiquettes de cluster pour le corpus PA
print("Cluster Labels pour le corpus PA:")
print(cluster_labels_trained_PA)

# Afficher les étiquettes de cluster pour le corpus PB
print("\nCluster Labels pour le corpus PB:")
print(cluster_labels_trained_PB)

# Afficher les étiquettes de cluster pour le corpus PC
print("\nCluster Labels pour le corpus PC:")
print(cluster_labels_trained_PC)

# Afficher les étiquettes de cluster pour le corpus PD
print("\nCluster Labels pour le corpus PD:")
print(cluster_labels_trained_PD)

# Afficher les étiquettes de cluster pour le corpus PE
print("\nCluster Labels pour le corpus PE:")
print(cluster_labels_trained_PE)


/home/olivier/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fa6c99cf1a0>
Traceback (most recent call last):
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
   

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fa6c99cf1a0>
Traceback (most recent call last):
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no 

Cluster Labels pour le corpus PA:
[2 0 0 0 0]

Cluster Labels pour le corpus PB:
[0 1 2 0 0]

Cluster Labels pour le corpus PC:
[2 0 0 0 0]

Cluster Labels pour le corpus PD:
[0 0 0 0 0]

Cluster Labels pour le corpus PE:
[1 2 2 0 0]


## 3.4 k-means avec Doc2Vec

In [27]:
from sklearn.cluster import KMeans
import numpy as np

# Concaténer les vecteurs des différents corpus
all_vectors = np.vstack([vectors_doc2vec_original, vectors_doc2vec_PA, vectors_doc2vec_PB, vectors_doc2vec_PC, vectors_doc2vec_PD, vectors_doc2vec_PE])

# Définir le nombre de clusters (vous pouvez ajuster ce nombre en fonction de vos besoins)
num_clusters = 6

# Appliquer l'algorithme K-Means
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(all_vectors)

# Ajouter les étiquettes de cluster aux vecteurs
all_vectors_with_labels = np.hstack([all_vectors, kmeans.labels_.reshape(-1, 1)])

# Afficher les centres des clusters
print("\nCentres des clusters:")
print(kmeans.cluster_centers_)

# Extraire les étiquettes de cluster pour chaque corpus
labels_original = kmeans.labels_[:len(vectors_doc2vec_original)]
labels_PA = kmeans.labels_[len(vectors_doc2vec_original):len(vectors_doc2vec_original) + len(vectors_doc2vec_PA)]
labels_PB = kmeans.labels_[len(vectors_doc2vec_original) + len(vectors_doc2vec_PA):len(vectors_doc2vec_original) + len(vectors_doc2vec_PA) + len(vectors_doc2vec_PB)]
labels_PC = kmeans.labels_[len(vectors_doc2vec_original) + len(vectors_doc2vec_PA) + len(vectors_doc2vec_PB):len(vectors_doc2vec_original) + len(vectors_doc2vec_PA) + len(vectors_doc2vec_PB) + len(vectors_doc2vec_PC)]
labels_PD = kmeans.labels_[len(vectors_doc2vec_original) + len(vectors_doc2vec_PA) + len(vectors_doc2vec_PB) + len(vectors_doc2vec_PC):len(vectors_doc2vec_original) + len(vectors_doc2vec_PA) + len(vectors_doc2vec_PB) + len(vectors_doc2vec_PC) + len(vectors_doc2vec_PD)]
labels_PE = kmeans.labels_[len(vectors_doc2vec_original) + len(vectors_doc2vec_PA) + len(vectors_doc2vec_PB) + len(vectors_doc2vec_PC) + len(vectors_doc2vec_PD):]

# Afficher les étiquettes de cluster pour chaque corpus
print("\nCluster Labels pour le corpus original:")
print(labels_original)

print("\nCluster Labels pour le corpus PA:")
print(labels_PA)

print("\nCluster Labels pour le corpus PB:")
print(labels_PB)

print("\nCluster Labels pour le corpus PC:")
print(labels_PC)

print("\nCluster Labels pour le corpus PD:")
print(labels_PD)

print("\nCluster Labels pour le corpus PE:")
print(labels_PE)

/home/olivier/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fa6c99cd800>
Traceback (most recent call last):
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/olivier/anaconda3/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
   


Centres des clusters:
[[-6.47747889e-02 -1.91159286e-02 -8.31584726e-03 -5.48813790e-02
   1.87961888e-02  2.92376038e-02  3.28550786e-02  6.52175993e-02
  -7.95004070e-02  5.10210618e-02  1.86022259e-02  5.19129410e-02
   1.44209377e-02 -6.38466701e-02  2.80806497e-02 -4.19065356e-02
   4.10488341e-03 -3.52052320e-03 -8.98423791e-02 -6.00960441e-02
  -7.04178289e-02 -8.71089194e-03 -1.74063481e-02  3.92489973e-03
  -1.77427437e-02  2.58704461e-02 -6.14321753e-02 -6.57376796e-02
  -4.44466919e-02 -1.46508776e-02  6.67144954e-02  4.12070304e-02
  -6.15302846e-02 -2.49910280e-02 -2.49719582e-02  8.03912729e-02
   5.39727602e-03 -4.62970957e-02  4.31538671e-02 -4.30148169e-02
   1.53735038e-02  5.95710129e-02  5.25323907e-03 -5.00948429e-02
  -1.12063054e-03  6.53213486e-02  3.49847451e-02 -1.95921469e-03
   5.01971617e-02 -2.00924780e-02]
 [-2.30378918e-02 -1.85350962e-02 -2.42593735e-02  1.26176337e-02
   1.11737242e-02  1.42034516e-03 -1.67665947e-02  1.86955556e-03
  -3.62652130e-02 